# Qubit rotation

This tutorial demonstrates the very basic working principles of openqml for qubit-based backends. We only look at a single quantum function consisting of a single-qubit circuit. The task is to optimize two rotation gates in order to flip the qubit from state $|0\rangle$ to state $|1\rangle$. 

First we need to import OpenQML, as well as OpenQML's version of NumPy. This allows us to automatically compute gradients for functions that manipulate NumPy arrays, including quantum functions. We call this NumPy version `onp` in case we need it alongside the original version.

In [ ]:
import openqml as qm
from openqml import numpy as np
from openqml.optimize import GradientDescentOptimizer, AdagradOptimizer

Next, create a projecq simulator as a "device" to run the quantum node. We only need a single quantum wire.

In [2]:
dev1 = qm.device('projectq.simulator', wires=1)

## Defining the quantum function

We define a quantum function called "circuit". 

In [3]:
@qm.qfunc(dev1)
def circuit(variables):
    
    qm.RX(variables[0], [0])
    qm.RY(variables[1], [0])
    
    return qm.expectation.PauliZ(0)

This function uses openqml to run the following quantum circuit:

<img src="figures/rotation_circuit.png">

Starting with a qubit in the ground state, 

$$ |0\rangle = \begin{pmatrix}1 \\ 0 \end{pmatrix}, $$

we first rotate the qubit around the x-axis by 
$$R_x(w_0) = e^{-iw_0 X /2} = 
\begin{pmatrix} \cos \frac{w_0}{2} &  -i \sin \frac{w_0}{2} \\  
                -i \sin \frac{w_0}{2} &  \cos \frac{w_0}{2} 
\end{pmatrix}, $$ 
               
and then around the y-axis by 
$$ R_y(w_1) = e^{-i w_1 Y/2} = 
\begin{pmatrix} \cos \frac{w_1}{2} &  - \sin \frac{w_1}{2} \\  
                \sin \frac{w_1}{2} &  \cos \frac{w_1}{2} 
\end{pmatrix}. $$ 

After these operations the qubit is in the state

$$ | \psi \rangle = R_y(w_0) R_x(w_1) | 0 \rangle $$

Finally, we measure the expectation $\langle\psi|Z|\psi\rangle$ of the Pauli-Z operator 
$$Z = 
\begin{pmatrix} 1 &  0 \\  
                0 & -1 
\end{pmatrix}. $$ 


Depending on the circuit parameters $w_1$ and $w_2$, the output expectation lies between $1$ (if $\left|\psi\right\rangle=\left|0\right\rangle$) and $-1$ (if $|\psi\rangle=|1\rangle$).

## Defining the objective

Next, we define a cost. Here, the cost is directly the expectation of the PauliZ measurement, so that the cost is trivially the output of the circuit.

In [4]:
def objective(variables):
    return circuit(variables)

With this objective, the optimization procedure is supposed to find the weights that rotate the qubit from the ground state 

 <img src="figures/bloch_before.png" width="250"> 
 
 to the excited state
 
 <img src="figures/bloch_after.png" width="250">
 
 The rotation gates give the optimization landscape a trigonometric shape with four global minima and five global maxima.
 
 <img src="figures/optlandscape.png" width="450">

## Optimization

The initial values of the x- and y-rotation parameters $w_1, w_2$ are set to near-zero. This corresponds to identity gates, in other words, the circuit leaves the qubit in the ground state.

In [5]:
variables_init = np.array([0.01, 0.01])
variables_init

array([ 0.01,  0.01])

The value of the objective at the initial point is close to $1$.

In [6]:
objective(variables_init)

0.9999000033332889

We choose a simple [Gradient Descent Optimizer](../API/optimize.rst#openqml.optimize.GradientDescentOptimizer) and update the weights for 10 steps. The final parameters correspond to a $Z$ expectation of nearly $-1$, which means that the qubit is flipped.

In [7]:
o = GradientDescentOptimizer(0.5)

variables = variables_init
for it in range(100):
    weights = o.step(objective, variables)
    if it % 5 == 0:
        print('Objective after step {:5d}: {: .7f}'.format(it, objective(weights)) )

print()
print('Optimized rotation angles:', variables)

Objective after step     0:  0.9997750
Objective after step     5:  0.9997750
Objective after step    10:  0.9997750
Objective after step    15:  0.9997750
Objective after step    20:  0.9997750
Objective after step    25:  0.9997750
Objective after step    30:  0.9997750
Objective after step    35:  0.9997750
Objective after step    40:  0.9997750
Objective after step    45:  0.9997750
Objective after step    50:  0.9997750
Objective after step    55:  0.9997750
Objective after step    60:  0.9997750
Objective after step    65:  0.9997750
Objective after step    70:  0.9997750
Objective after step    75:  0.9997750
Objective after step    80:  0.9997750
Objective after step    85:  0.9997750
Objective after step    90:  0.9997750
Objective after step    95:  0.9997750

Optimized rotation angles: [ 0.01  0.01]


Starting at a different offset, we train another optimizer called [Adagrad](../API/optimize.rst#openqml.optimize.Adagrad), which improves on gradient descent.

In [8]:
variables_init = np.array([-0.01, 0.01])
print('Initial rotation angles:', variables_init)

o = AdagradOptimizer(0.5)

variables = variables_init
for it in range(100):
    variables = o.step(objective, variables)
    if it % 5 == 0:
        print('Objective after step {:5d}: {: .7f}'.format(it+1, objective(variables)) )

print()
print('Optimized rotation angles:', variables)

Initial rotation angles: [-0.01  0.01]
Objective after step     1:  0.7617043
Objective after step     6:  0.0000072
Objective after step    11:  0.0000000
Objective after step    16:  0.0000000
Objective after step    21:  0.0000000
Objective after step    26: -0.0000000
Objective after step    31: -0.0000000
Objective after step    36: -0.0000000
Objective after step    41: -0.0000000
Objective after step    46: -0.0000081
Objective after step    51: -0.0024196
Objective after step    56: -0.3870690
Objective after step    61: -0.9926985
Objective after step    66: -0.9999724
Objective after step    71: -0.9999999
Objective after step    76: -1.0000000
Objective after step    81: -1.0000000
Objective after step    86: -1.0000000
Objective after step    91: -1.0000000
Objective after step    96: -1.0000000

Optimized rotation angles: [ -3.14159265e+00   2.94207915e-11]


 Adagrad and gradient descent find the same minimum, and, since neither has information on second order derivatives, both take a detour through a saddle point. However, Adagrad takes considerably fewer steps.
 
 <img src="figures/gd_vs_adag_qubit.png" width="450">